In [1]:
import pandas as pd
import folium
import geopandas as gpd
from folium.features import GeoJsonTooltip
import fiona

files = [
    "data/patient-medical-record-24.xlsx",
    "data/patient-medical-record-23.xlsx",
    "data/patient-medical-record-18.xlsx",
    "data/patient-medical-record-17.xlsx",
    "data/patient-medical-record-16.xlsx",
    "data/patient-medical-record-15.xlsx",
    "data/patient-medical-record-14.xlsx"
]

# Read and concatenate all DataFrames
df_list = [pd.read_excel(file) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Load the data
# df = pd.read_excel("data/patient-medical-record-2017.xlsx")

# Drop rows with missing data
df = df.dropna(subset=["patients_disposition", "patients_disposition_location"])
released_df = df[df["patients_disposition"] == "Released"].copy()
released_df = released_df[["patients_disposition", "patients_disposition_location"]]

## Extract City and County Info

In [3]:
# --- Load municipality-county pairs ---
municipality_county = pd.read_csv("data/wa_municipality_county_pairs.csv")
municipality_county["Municipality"] = municipality_county["Municipality"].str.lower().str.strip()
municipality_county["County"] = municipality_county["County"].str.lower().str.strip()

# Create a dictionary for quick lookup of county by municipality
city_to_county = dict(zip(municipality_county["Municipality"], municipality_county["County"]))

# --- Extract city and county from location field ---
def extract_city_county(location, city_county_dict):
    location = str(location).lower()
    for city, county in city_county_dict.items():
        if city in location:
            return city, county
    return None, None

# Apply extraction to create both city and county columns
released_df[["disposition_city", "disposition_county"]] = released_df["patients_disposition_location"].apply(
    lambda x: pd.Series(extract_city_county(x, city_to_county))
)

released_df.to_csv("14to24/release_city_county_14to24.csv", index=False)

## Release Map by City

In [ ]:
# --- Load GDB of city boundaries ---
gdb_path = "CityLimits/CityLimits.gdb"
layer_name = fiona.listlayers(gdb_path)[0]
gdf = gpd.read_file(gdb_path, layer=layer_name)
gdf['CityName'] = gdf['CityName'].str.lower().str.strip()

df = pd.read_csv("14to24/release_city_county_14to24.csv")
df['disposition_city'] = df['disposition_city'].str.lower()
city_counts = df['disposition_city'].value_counts().reset_index()
city_counts.columns = ['CityName', 'count']

# --- Merge counts with city geometries ---
merged = gdf.merge(city_counts, on="CityName", how="left").fillna({'count': 0})
nonzero = merged[merged['count'] > 0].copy()
nonzero = nonzero.to_crs(epsg=4326)
nonzero = nonzero.drop(columns=["LastUpdate"], errors="ignore")

# --- Plot interactive map ---
m = folium.Map(location=[47.6062, -122.3321], zoom_start=9, tiles="CartoDB positron")

# --- Add fixed title that stays on top ---
title_html = '''
<div style="position: fixed; 
            top: 10px; 
            left: 50%; 
            transform: translateX(-50%); 
            z-index: 1000;
            background-color: rgba(255, 255, 255, 0.9);
            padding: 10px 20px;
            border-radius: 10px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.3);">
    <h3 style="margin: 0; font-size: 20px; font-weight: bold; color: #333;">
        Release From 2014 To 2024 Grouped by City
    </h3>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add choropleth
folium.Choropleth(
    geo_data=nonzero,
    data=nonzero,
    columns=["CityName", "count"],
    key_on="feature.properties.CityName",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Number of Releases",
    nan_fill_color="white",
).add_to(m)

# Optional: Add tooltip
folium.GeoJson(
    nonzero,
    tooltip=GeoJsonTooltip(
        fields=["CityName", "count"],
        aliases=["City", "Releases"],
        localize=True,
        sticky=True,
        labels=True
    ),
    style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.3},
).add_to(m)

# Save
m.save("14to24/release_city_map_14to24.html")

## Release Map by County

In [7]:
# --- Load county boundaries ---
county_gdf = gpd.read_file("CountyBoundaries/wa_counties.shp")
county_gdf = county_gdf.to_crs(epsg=4326)  # Ensure WGS84 for web mapping

# --- Load release county data ---
df = pd.read_csv("14to24/release_city_county_14to24.csv")

# --- Aggregate data by county (capitalize county names to match shapefile) ---
df['disposition_county'] = df['disposition_county'].str.capitalize()
county_counts = df['disposition_county'].value_counts().reset_index()
county_counts.columns = ['County', 'count']

# --- Merge geographic and count data ---
merged = county_gdf.merge(county_counts, left_on='NAME', right_on='County', how='left')
merged['count'] = merged['count'].fillna(0).astype(int)

# --- Filter to only counties with data ---
counties_with_data = merged[merged['count'] > 0].copy()

# --- Initialize map centered on Washington State ---
m = folium.Map(location=[47.7511, -120.7401], zoom_start=7, tiles="CartoDB positron")

# --- Add fixed title that stays on top ---
title_html = '''
<div style="position: fixed; 
            top: 10px; 
            left: 50%; 
            transform: translateX(-50%); 
            z-index: 1000;
            background-color: rgba(255, 255, 255, 0.9);
            padding: 10px 20px;
            border-radius: 10px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.3);">
    <h3 style="margin: 0; font-size: 20px; font-weight: bold; color: #333;">
        Release From 2014 To 2024 Grouped by County
    </h3>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# --- Add Choropleth layer only for counties WITH data ---
if len(counties_with_data) > 0:
    folium.Choropleth(
        geo_data=counties_with_data,
        data=counties_with_data,
        columns=['NAME', 'count'],
        key_on='feature.properties.NAME',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name='Number of Release Records',
        highlight=True,
        line_color='black'
    ).add_to(m)
    
    # --- Add interactive tooltips only for counties with data ---
    folium.GeoJson(
        counties_with_data,
        tooltip=GeoJsonTooltip(
            fields=['NAME', 'count'],
            aliases=['County:', 'Release Records:'],
            localize=True,
            sticky=True,
            labels=True,
            style="""
                background-color: white;
                border: 2px solid black;
                border-radius: 3px;
                box-shadow: 3px;
            """
        ),
        style_function=lambda x: {
            'fillOpacity': 0,
            'color': 'transparent',
            'weight': 0
        },
        highlight_function=lambda x: {
            'weight': 3,
            'color': 'darkblue',
            'fillOpacity': 0
        }
    ).add_to(m)

# --- Save the map ---
m.save("14to24/release_county_map_14to24.html")